<a href="https://colab.research.google.com/github/JS0501/ESAA_YB3/blob/main/YB0502_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C_%EB%B6%84%EB%A5%982.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Lightgbm

* 데이터 '와인품질분류.zip'입니다.

1. 필요한 라이브러리를 가져옵니다.

In [ ]:
import pandas as pd
import lightgbm as lgbm

2. 데이터를 불러옵니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/train.csv')
test = pd.read_csv('/content/drive/MyDrive/test.csv')
smpl_sub = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')

In [ ]:
train.head()

,index,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,white
1,1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,red
2,2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,white
3,3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,white
4,4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,white


In [ ]:
test.head()

,index,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,0,9.0,0.31,0.48,6.6,0.043,11.0,73.0,0.99380,2.90,0.38,11.6,white
1,1,13.3,0.43,0.58,1.9,0.070,15.0,40.0,1.00040,3.06,0.49,9.0,red
2,2,6.5,0.28,0.27,5.2,0.040,44.0,179.0,0.99480,3.19,0.69,9.4,white
3,3,7.2,0.15,0.39,1.8,0.043,21.0,159.0,0.99480,3.52,0.47,10.0,white
4,4,6.8,0.26,0.26,2.0,0.019,23.5,72.0,0.99041,3.16,0.47,11.8,white


3. 변수를 변환합니다.

In [ ]:
# type에는 white와 red 두 종류가 있습니다.
# 각각 0,1로 변환합니다.
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['type'] = le.fit_transform(train['type'])
test['type'] = le.fit_transform(test['type'])

In [ ]:
train.head()

,index,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,1
1,1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,0
2,2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,1
3,3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,1
4,4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,1


In [ ]:
test.head()

,index,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,0,9.0,0.31,0.48,6.6,0.043,11.0,73.0,0.99380,2.90,0.38,11.6,1
1,1,13.3,0.43,0.58,1.9,0.070,15.0,40.0,1.00040,3.06,0.49,9.0,0
2,2,6.5,0.28,0.27,5.2,0.040,44.0,179.0,0.99480,3.19,0.69,9.4,1
3,3,7.2,0.15,0.39,1.8,0.043,21.0,159.0,0.99480,3.52,0.47,10.0,1
4,4,6.8,0.26,0.26,2.0,0.019,23.5,72.0,0.99041,3.16,0.47,11.8,1


In [ ]:
# 모델에 입력하기 전 데이터를 정형합니다.
# train_x : train의 'index', 'quality' 피처를 제거하고 저장합니다.
# train_y : train의 'quality'를 저장합니다.
# test_x : test의 'index'를 제거하고 저장합니다.

train_x = train.drop(['index','quality'],axis = 1)
train_y = train['quality']
test_x = test.drop(['index'],axis = 1)

In [ ]:
train_x.shape, train_y.shape, test_x.shape

((5497, 12), (5497,), (1000, 12))

In [ ]:
train_x.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,1
1,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,0
2,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,1
3,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,1
4,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,1


In [ ]:
train_y.head()

,quality
0,5
1,5
2,5
3,6
4,6


In [ ]:
test_x.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,9.0,0.31,0.48,6.6,0.043,11.0,73.0,0.99380,2.90,0.38,11.6,1
1,13.3,0.43,0.58,1.9,0.070,15.0,40.0,1.00040,3.06,0.49,9.0,0
2,6.5,0.28,0.27,5.2,0.040,44.0,179.0,0.99480,3.19,0.69,9.4,1
3,7.2,0.15,0.39,1.8,0.043,21.0,159.0,0.99480,3.52,0.47,10.0,1
4,6.8,0.26,0.26,2.0,0.019,23.5,72.0,0.99041,3.16,0.47,11.8,1


4. LightGBM 모델을 생성 및 훈련합니다.

In [ ]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier()
lgbm.fit(train_x,train_y)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000218 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1538
[LightGBM] [Info] Number of data points in the train set: 5497, number of used features: 12
[LightGBM] [Info] Start training from score -5.353861
[LightGBM] [Info] Start training from score -3.386211
[LightGBM] [Info] Start training from score -1.123105
[LightGBM] [Info] Start training from score -0.822089
[LightGBM] [Info] Start training from score -1.783246
[LightGBM] [Info] Start training from score -3.588077
[LightGBM] [Info] Start training from score -7.002520
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits w

LGBMClassifier()

5. 생성한 모델로 예측 데이터를 생성합니다.

In [ ]:
lgbm_pred = lgbm.predict(test_x)

6. 제출파일을 생성합니다.

In [ ]:
smpl_sub.head()

,index,quality
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [ ]:
#서브미션 파일의 quality 변수에 예측값을 대입합니다.
smpl_sub['quality']=lgbm_pred

In [ ]:
smpl_sub.head()

,index,quality
0,0,5
1,1,5
2,2,6
3,3,5
4,4,6


In [ ]:
# csv 파일로 저장합니다.
smpl_sub.to_csv('submission.csv', index=False)

# 2. XGBoost

In [ ]:
# 데이터 다운로드 링크로 데이터를 코랩에 불러옵니다.

!wget 'https://bit.ly/3i4n1QB'

import zipfile
with zipfile.ZipFile('3i4n1QB', 'r') as existing_zip:
    existing_zip.extractall('data')

--2025-05-02 10:14:30--  https://bit.ly/3i4n1QB
Resolving bit.ly (bit.ly)... 67.199.248.10, 67.199.248.11
Connecting to bit.ly (bit.ly)|67.199.248.10|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw [following]
--2025-05-02 10:14:30--  https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw
Resolving drive.google.com (drive.google.com)... 142.250.141.113, 142.250.141.139, 142.250.141.100, ...
Connecting to drive.google.com (drive.google.com)|142.250.141.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw&export=download [following]
--2025-05-02 10:14:30--  https://drive.usercontent.google.com/download?id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)...

In [ ]:
# 라이브러리 및 데이터 불러오기
import pandas as pd
from xgboost import XGBClassifier

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

train.drop('index',axis = 1 ,inplace =True)
test.drop('index',axis = 1 ,inplace =True)

In [ ]:
# 데이터 확인
train.head()

,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,white
1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,red
2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,white
3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,white
4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,white


In [ ]:
test.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,9.0,0.31,0.48,6.6,0.043,11.0,73.0,0.99380,2.90,0.38,11.6,white
1,13.3,0.43,0.58,1.9,0.070,15.0,40.0,1.00040,3.06,0.49,9.0,red
2,6.5,0.28,0.27,5.2,0.040,44.0,179.0,0.99480,3.19,0.69,9.4,white
3,7.2,0.15,0.39,1.8,0.043,21.0,159.0,0.99480,3.52,0.47,10.0,white
4,6.8,0.26,0.26,2.0,0.019,23.5,72.0,0.99041,3.16,0.47,11.8,white


In [ ]:
# 원핫 인코딩 (pd.get_dummies())
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [ ]:
# 모델 정의
XGB_clf = XGBClassifier()

In [ ]:
# Create a LabelEncoder object
le = LabelEncoder()

# Fit the encoder to your target variable and transform it
y_encoded = le.fit_transform(y)

# Now use y_encoded for training your model
XGB_clf.fit(X, y_encoded)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [ ]:
# 모델 학습
# X 는 train에서 quality 를 제외한 모든 변수
# y 는 train의 qulity 변수
X = train.drop('quality',axis = 1)
y = train['quality']

# fit 메소드를 이용해 모델 학습
XGB_clf.fit(X,y_encoded)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [ ]:
# predict 메소드와 원핫 인코딩한 test 데이터를 이용해 품질 예측
XGB_clf.predict(test)

array([2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 2, 2, 3, 3, 2, 2, 3, 4,
       2, 4, 3, 2, 2, 2, 2, 2, 4, 3, 2, 3, 2, 3, 2, 2, 3, 3, 3, 2, 2, 3,
       3, 3, 2, 4, 1, 3, 2, 2, 3, 1, 2, 3, 3, 2, 2, 3, 2, 3, 3, 1, 3, 3,
       2, 2, 3, 2, 2, 2, 2, 3, 3, 2, 3, 3, 3, 4, 3, 3, 2, 2, 3, 3, 2, 3,
       4, 3, 3, 3, 3, 3, 3, 4, 3, 4, 3, 3, 2, 3, 3, 2, 2, 3, 3, 3, 4, 2,
       3, 3, 3, 2, 2, 4, 3, 4, 4, 3, 3, 2, 3, 3, 2, 2, 2, 3, 3, 4, 3, 2,
       3, 2, 2, 2, 3, 4, 2, 2, 2, 3, 2, 2, 2, 2, 4, 3, 5, 4, 2, 5, 3, 2,
       3, 2, 2, 2, 3, 2, 2, 3, 3, 4, 3, 2, 3, 3, 2, 3, 2, 2, 3, 2, 3, 3,
       3, 2, 3, 2, 3, 3, 4, 4, 2, 3, 3, 3, 2, 3, 3, 3, 3, 3, 2, 3, 4, 2,
       3, 2, 3, 2, 3, 3, 2, 2, 3, 3, 3, 3, 4, 2, 3, 2, 2, 3, 3, 3, 3, 4,
       3, 2, 3, 3, 4, 3, 3, 2, 3, 2, 3, 4, 4, 3, 3, 3, 3, 3, 3, 4, 3, 3,
       3, 3, 4, 3, 3, 4, 2, 3, 2, 3, 4, 4, 3, 3, 3, 4, 4, 3, 4, 2, 2, 4,
       2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 3, 2, 4, 4, 2, 3, 3, 3, 3, 3, 4, 4,
       2, 3, 3, 3, 2, 2, 2, 3, 2, 4, 5, 2, 2, 5, 4,

In [ ]:
# sample_submission.csv 파일을 불러와 예측된 값으로 채워 주기
submission = pd.read_csv('data/sample_submission.csv')
submission['quality'] = XGB_clf.predict(test)

In [ ]:
# 정답파일 내보내기
submission.to_csv('xgb_pred.csv',index = False)

# 3. 보팅

In [ ]:
# 데이터 다운로드 링크로 데이터를 코랩에 불러옵니다.

!wget 'https://bit.ly/3i4n1QB'

import zipfile
with zipfile.ZipFile('3i4n1QB', 'r') as existing_zip:
    existing_zip.extractall('data')

--2025-05-02 10:21:10--  https://bit.ly/3i4n1QB
Resolving bit.ly (bit.ly)... 67.199.248.11, 67.199.248.10
Connecting to bit.ly (bit.ly)|67.199.248.11|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw [following]
--2025-05-02 10:21:11--  https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw
Resolving drive.google.com (drive.google.com)... 142.250.141.100, 142.250.141.102, 142.250.141.139, ...
Connecting to drive.google.com (drive.google.com)|142.250.141.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw&export=download [following]
--2025-05-02 10:21:11--  https://drive.usercontent.google.com/download?id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)...

In [ ]:
# 라이브러리 및 데이터 불러오기
import pandas as pd
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

train.drop('index',axis = 1 ,inplace =True)
test.drop('index',axis = 1 ,inplace =True)

In [ ]:
# 데이터 확인
train.head()

,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,white
1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,red
2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,white
3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,white
4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,white


In [ ]:
test.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,9.0,0.31,0.48,6.6,0.043,11.0,73.0,0.99380,2.90,0.38,11.6,white
1,13.3,0.43,0.58,1.9,0.070,15.0,40.0,1.00040,3.06,0.49,9.0,red
2,6.5,0.28,0.27,5.2,0.040,44.0,179.0,0.99480,3.19,0.69,9.4,white
3,7.2,0.15,0.39,1.8,0.043,21.0,159.0,0.99480,3.52,0.47,10.0,white
4,6.8,0.26,0.26,2.0,0.019,23.5,72.0,0.99041,3.16,0.47,11.8,white


In [ ]:
# 원핫 인코딩 (pd.get_dummies())
pd.get_dummies(train)
pd.get_dummies(test)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type_red,type_white
0,9.0,0.31,0.48,6.60,0.043,11.0,73.0,0.99380,2.90,0.38,11.6,False,True
1,13.3,0.43,0.58,1.90,0.070,15.0,40.0,1.00040,3.06,0.49,9.0,True,False
2,6.5,0.28,0.27,5.20,0.040,44.0,179.0,0.99480,3.19,0.69,9.4,False,True
3,7.2,0.15,0.39,1.80,0.043,21.0,159.0,0.99480,3.52,0.47,10.0,False,True
4,6.8,0.26,0.26,2.00,0.019,23.5,72.0,0.99041,3.16,0.47,11.8,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,7.1,0.59,0.02,2.30,0.082,24.0,94.0,0.99744,3.55,0.53,9.7,True,False
996,8.7,0.15,0.30,1.60,0.046,29.0,130.0,0.99420,3.22,0.38,9.8,False,True
997,8.8,0.66,0.26,1.70,0.074,4.0,23.0,0.99710,3.15,0.74,9.2,True,False
998,7.0,0.42,0.19,2.30,0.071,18.0,36.0,0.99476,3.39,0.56,10.9,True,False


In [ ]:
# 모델 정의 (LightGBM, XGBoost, Random Forest)
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

lgbm = LGBMClassifier()
xgb = XGBClassifier()
rf = RandomForestClassifier()

# VotingClassifier 정의
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(estimators = [('lgbm',lgbm),('xgb',xgb),('rf',rf)],voting = 'soft')

In [ ]:
# Create a LabelEncoder object
le = LabelEncoder()

# Fit the encoder to your target variable and transform it
train_y_encoded = le.fit_transform(train_y)

In [ ]:
train_x = pd.get_dummies(train_x, columns=['type'])
test_x = pd.get_dummies(test_x, columns=['type'])

# Also apply one-hot encoding to the 'type' column in X
X = pd.get_dummies(X, columns=['type'])

In [ ]:
# 모델 학습
lgbm.fit(train_x,train_y_encoded)
xgb.fit(train_x,train_y_encoded)
rf.fit(train_x,train_y_encoded)
voting_clf.fit(train_x,train_y_encoded)

# X 는 train에서 quality 를 제외한 모든 변수
# y 는 train의 qulity 변수
X = train.drop('quality',axis = 1)
y = train['quality']

# fit 메소드를 이용해 모델 학습
voting_clf.fit(X,y)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000613 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1540
[LightGBM] [Info] Number of data points in the train set: 5497, number of used features: 13
[LightGBM] [Info] Start training from score -5.353861
[LightGBM] [Info] Start training from score -3.386211
[LightGBM] [Info] Start training from score -1.123105
[LightGBM] [Info] Start training from score -0.822089
[LightGBM] [Info] Start training from score -1.783246
[LightGBM] [Info] Start training from score -3.588077
[LightGBM] [Info] Start training from score -7.002520
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits w

ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: type: object

In [ ]:
# predict 메소드와 원핫 인코딩한 테스트 데이터를 이용해 품질 예측
lgbm.predict(test)
xgb.predict(test)
rf.predict(test)
voting_clf.predict(test)